<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/blob/main/Documentation/InteraktiveCamera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grundlegende Idee**
Die Idee ist ein Zwei-Kamera-System zu einem Ein-Kamera-System umzugestalten. Dabei soll die erste Kamera auf einen Sprecher und die zweite Kamera auf ein Blatt Papier gerichtet sein. Sobald der Sprecher nun seinen Zeigefinger auf das Papier legt, wird dieses Kamerabild als Bild in Bild (pip) der ersten Kamerasicht angezeigt.

Ein alternativer und womöglich attraktiverer Anwendungszeck ist, das System als Ein-Kamera-System in Verbindung mit einer Bildschirmübertragung zu verwenden: 
Studenten, wie beispielsweise Tutoren, haben oftmals den Bedarf daran etwas graphisch darstellen zu wollen, falls etwas in einer Präsentation den Zuhörern nicht ganz klar geworden ist. In einem Seminarraum ist dies mit einer Tafel schnell umgesetzt, in der jetztigen Corona Pandemie von 2020, gestaltet sich dies für viele ohne benötigtes Equipment wie einem Grafiktablett eher schwer.
So liegt es nahe Papier und Stift anstelle des Grafiktablets zu verwenden und dieses Bild als pip in der Bildschirmübertragung anzuzeigen.




## Required Hardware

The system should be as simple as possible and not require any special hardware. This means that a cell phone camera, which delivers a camera feed to the PC via apps such as [Droid Cam](http://www.dev47apps.com/) on Android and iOS, should be sufficient. Accordingly, it should also be possible to display this cell phone camera image within a screen transmission in order to support what is shown on the screen with hand drawings, for example.

## Funktionen
Wie bereits genannt wird der Nutzer zwei Bildinputs auswählen und festlegen müssen welche der Inputs das einzublendende Bild enthält. Dem Nutzer soll es möglich sein durch Zeigen mit dem Zeigefinger die Kamera zu Aktivieren und den Zeigefinger als Mittelpunkt des Bildes zu sehen. Durch eine "Raus-Zoom" Geste wie man sie von Smartphones kennt, wird ein digitaler Zoom ausführbar sein.

## Programmcode
Jeglicher Programmcode ist in dem folgende Repository zu finden:
[Medienverarbeitung Gruppe E - Interactive Camera](https://github.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system)

## Vorgehen im Projekt
Das Projekt wird als Projekttagebuch dokumentiert. Hier werden alle Ideen, Ansätze und Ergebnisse festgehalten. Zu Abschluss des Projekts wird das Gesamtergebnis noch einmal gesondert und aufbereitet präsentiert. 


# Projekttagebuch

## Erster Ansatz und Ergebnisse 
*11.11.20*

Zu erst kümmern wir uns um die Erkennung eines zeigenden Zeigefingers. Dafür trainieren wir ein Keras Modell mit Bildern, welche zeigende Zeigefinger (Bild 1), weiße (karrierte, linierte, blanko) Papiere (Bild 2) und andere Gesten wie flache Hände und Ähnliches (Bild 3) unterscheiden können soll.

![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/11.11.20/zu_unterscheidende_Objekte.png)

Das damit antrainierte Modell erkennt das gezeigte Bild und sortiert es der Rubrik ein, welche mit der höchsten Konfidenz übereinstimmt.

Die Verarbeitung des eingehenden Videostreams wird durch die Python-Bibliothek "[*OpenCV*](https://pypi.org/project/opencv-python/)" ermöglicht.

Durch diesen Ansatz konnten bisher erste Erfolge und enstehende Probleme verzeichnet werden

###Probleme
Es bisher Störfaktoren wie eine Computer-Maus (Bild 6) fälschlicherweise als ein Zeigen erkannt. Jedoch war es auch möglich in einer kontrollierten Umgebung einen Zeigefinger (Bild 4) von einem Blatt Papier (Bild 5) zu unterscheiden.
![alt text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/11.11.20/Ergebnisse.png)



##Verwurf vorheriger Ergebnisse und entdeckung neuer Technologien
*18.11.2020*

Eine Objekterkennung durch ein neurales Netzwerk bedarf einer ausgesprochen Großen Datenmenge, welche kuriert wurden, um sie auf Relevante objekte zu reduzieren. Dies heißt in unserem Fall, dass wir einen für zwei Personen nicht erzeugbaren Datensatz an zeigenden Fingern und Hintergrundbildern benötigen würden, um akkurate Ergebnise zu erziehlen. Daher brauchen wir eine Alternative, welche mit einem realistischen Arbeitsaufwand zu bewältigen ist.

Nun stellt sich die Frage: Was macht einen zeigenden Finger aus? Oder noch simpler: Was macht eine Hand aus?

Wir als Menschen erkennen Objekte auf Distanzen primär über Form und Farbe. So entsteht die Idee, dass ein Computer ein Bild nach der Handfarbe eines Nutzers absuchen und diesen Bereich isolieren kann. Dies ist in "[*OpenCV*](https://pypi.org/project/opencv-python/)" über sogenannte Histogramme möglich. 
Die Grundlegende Idee ist die solche, dass anfangs die Hautfarbe des Nutzers durch eine simple Bildprobe ermittelt wird und so eine Hand (bzw. etwas Hautfarbenes) in einem Bild erkannt werden kann. Dabei besteht die Annahme, dass auch nur *eine* Hand im Bild zu sehen ist und keine weiteren mit der Hautfarbe übereinstimmenden Gegenstände im Bild sind.

![alt Text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/18.11.20/Messung_Hautfarbe.png "Bild 7: Messpunkte auf Hand")

**Bild 7:** Messpunkte auf Hand




In Bild 7 ist eine solche Messung zu sehen. Die dabei markierten Rechtecke werden zu einem Bild zusammengefügt und aus diesem entstehenden Bild ein Histogram errechnet. Hier zu sehen ist, wie diese Regionen markiert und für die Messung genutzt werden:


```python
def drawMeasuringRectangles(frame):
    """Draws 'amountOfMeasuringRectangles' Rectangles on the given frame and returns the modified image"""
    rows, cols, dontCare = frame.shape
    global amountOfMeasuringRectangles, xCoordinatesOfMeasuringRectangles_topLeft, yCoordinatesOfMeasuringRectangles_topLeft, xCoordinatesOfMeasuringRectangles_bottomRight, yCoordinatesOfMeasuringRectangles_bottomRight

    # position messure points of hand histogram
    xCoordinatesOfMeasuringRectangles_topLeft = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    yCoordinatesOfMeasuringRectangles_topLeft = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    # define shape of drawn small rectangles | here 10x10
    xCoordinatesOfMeasuringRectangles_bottomRight = xCoordinatesOfMeasuringRectangles_topLeft + 10
    yCoordinatesOfMeasuringRectangles_bottomRight = yCoordinatesOfMeasuringRectangles_topLeft + 10

    # draw calculated rectangles
    for i in range(amountOfMeasuringRectangles):
        cv2.rectangle(frame,
                      (yCoordinatesOfMeasuringRectangles_topLeft[i], xCoordinatesOfMeasuringRectangles_topLeft[i]),
                      (yCoordinatesOfMeasuringRectangles_bottomRight[i],
                       xCoordinatesOfMeasuringRectangles_bottomRight[i]),
                      (0, 255, 0), 1)

    return frame

  def createHandHistogram(frame):
    global xCoordinatesOfMeasuringRectangles_topLeft, yCoordinatesOfMeasuringRectangles_topLeft

    # convert cv2 bgr colorspace to hsv colorspace for easier handling
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # create new blank Region Of Interest matrix/image
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)

    # fill ROI with the sample rectangles
    for i in range(amountOfMeasuringRectangles):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[xCoordinatesOfMeasuringRectangles_topLeft[i]:
                                                    xCoordinatesOfMeasuringRectangles_topLeft[i] + 10,
                                          yCoordinatesOfMeasuringRectangles_topLeft[i]:
                                          yCoordinatesOfMeasuringRectangles_topLeft[i] + 10]

    # create a Hand histogram and normalize it
    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # remove noise and retun
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)
```
Das daraus entstehende Histogram wird verwendet, um auf das gegebene Bild bzw. den Videoframe einen *Back Projection* anzuwenden. Dabei stimmen hellere (weißere) Pixel mehr mit dem errechneten Histogram überein, während dunklere (schwarze) Pixel nicht über einstimmen und so eine andere Farbe vorweisen. Das daraus resultierende Bild ist in *Bild 8* zu sehen

![altText](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/18.11.20/Hand_Nach_Histogram_BackProjection.png)

**Bild 8:** Hand nach `cv2.calcBackProject( [...] )`

Offensichtlich ist dieses Bild noch sehr verrauscht und zu feingranular, als dass man daraus einfach einen nützlichen Informationsgewinn erzielen könnte. Aufgrund dessen werden mit einer simplen *Threshhold* Operation alle Pixel als unrelevant erachtet, welche einen zu geringen Weißwert aufweisen und so am ehesten einen falsch-positiven Pixel darstellen. Diese Übriggebliebenen Pixel werden abschließend jeweils mit einem weißen Kreis maskiert, um die verlorenen Pixel wett zu machen und das bild in ein simples schwarzweiß Bild, wie in Bild 9 zu sehen ist, umzuwandeln.

![alttext](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/18.11.20/SchwarzWei%C3%9F_Histogram_Backprojection.png)

**Bild 9:** Schwarzweiß Bild nach `cv2.threshhold( [...] )`

Diese Maske wird nun Abschließend genutzt, um eine im Optimalfall konvexe Hülle um die Hand zu legen. Mithilfe dieser Hülle berechnen wir nun den Mittelpunkt der Hand und zeichnen diesen ein. Dies ist der lilane Punkt in Bild 10.  

```python
#Beispielcode zum berechnen eines Mittelpunktes anhand von Konturen
[...]
contourList = getContoursFromMaskedImage(maskedHistogramImage)
  if contourList:
    maxCont = max(contourList, key=cv2.contourArea)
    centerOfMaxCont = getCenterCoordinatesOfContour(maxCont)
[...]


def getContoursFromMaskedImage(maskedHistogramImage):
    """Returns the contours of a given masked Image"""
    grayscaledMaskedHistogramImage = cv2.cvtColor(maskedHistogramImage, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(grayscaledMaskedHistogramImage, 0, 255, 0)
    cont, hierarchyDontCare = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cont
``` 

Zeigt diese Hand nun mit dem Finger auf etwas, so ist dies durch einen Störung der konvexen Hülle, als ein sogenanter *konvexer Defekt*, erkennbar.  Wir nutzen diesen Defekt, um mithilfe des Mittelpunkts der Hand den am weitesten sich auf der Kontur befindlichen Punkt zu ermitteln. Dabei besteht die Annahme, dass dieser Punkt die Fingerspitze repräsentiert. Der folgende Code zeigt eine solche Punktermittlung:

```python
hull = cv2.convexHull(maxCont, returnPoints=False)
            defects = cv2.convexityDefects(maxCont, hull)
            farthestPoint = getFarthestPointFromContour(defects, maxCont, centerOfMaxCont)

def getFarthestPointFromContour(defects, contour, centroid):
    """Returns the farthest point from a given centerpoint on a contour using defects"""
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            return tuple(contour[farthest_defect][0])
        else:
            return None
```
In Bild 10 sehen wir wie diese Punkte (hier gelb) genutzt werden um sie als Fingerspitze anzuzeigen. 

Jedoch durch diese Operationen auch rauschende Pixel wie in Bild 8 zu sehen ist hin und wieder immernoch als Teil der Hand erkannt und sorgen so für Probleme bei der Erkennung von konvexen Defekten. Dieses Problem gilt es zu lösen.


##Verbesserung der Fingerspitzenerkennung und Ausblick auf wiederverwendung einer KI

Zunächst ist die Idee, dass wir fehlgesetzte Punkte der Fingerspitze (Bild 10) als Rauschen identifizieren und so ignorieren können. Dies soll der Erkennbarkeit der Fingerspitze steigern und die Anzahl der falsch-positive Punkte verringern.

Dies ist umzusetzen indem man erst einen (rauschbehafteten) Satz an Punkten sammelt, von dort aus deren gemeinsamen Mittelpunkt errechnet und zuletzt alle weiteren Punkte, die nicht in einem gewissen Radius um diesen Mittelpunkt liegen, ignoriert. Dadurch erreicht man innerhalb weniger dutzend Bilder einen Rauschfreien Punktesatz wie diesem in Bild 11

![alt Text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/25.11.20/Finger_mit_Rauschen_Durch_Fehlgesetzte_Punkte.png)

**Bild 10:** Finger mit rauschenden Punkten

![alt Text](https://raw.githubusercontent.com/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/main/Documentation/Pictures/25.11.20/Finger_Ohne_Rauschen.png)

**Bild 11:** Finger ohne rauschende Punkte


Da nun die Fingerspitzenerkennung für alle weiteren Nutzungszwecke ausreichend funktioniert, gilt es nun das gleiche für die Erkennung des Handmittelpunktes durchzuführen. Die Rauschunterdrückung läuft auch hier analog zur vorherigen ab.



## Hinzufügen einer einfachen Gui, um den zu verwendenen Monitor und die Kamera auszuwählen

Damit der Nutzer später auch zwischen verschiedenen Monitoren und ggf. Kameras einfach wechseln kann, ohne das Programm anpassen zu müssen, wurde von uns eine SImple GUI entwicklet. Diese stellt zwei Dropdown Menüs bereit, welche wiederum alle angeschlossenen Monitore und Kameras auflisten und zwischen denen beliebig gewechselt werden kann.

Um diese möglichst einfach zu implementieren, verwenden wir eine Bibliothek namens TKInter. Diese bietet eine recht intuitive und einfache Möglichkeit, diverse Bedienelemente in einem Fenster anzuordnen und zu verwalten.

**Bild 12:** DropDown Menüs

##Ausblick auf die KI

Ursprünglich haben wir Tensorflow verwendet, um einen Zeigenden Finger von einem nichtzeigenden Finger zu unterscheiden. Jedoch stößt diese Herangehensweise schnell an seine Grenzen. Beispielsweise werden schwarze Hände nicht erkannt, da das Trainingsmaterial aus ausschließlich weißen Händen besteht. Jedoch ist es möglich durch die Verwendung von Bildern wie Bild 9 oder Bild 8 ein neutrales Grundbild zu schaffen. Das heißt, unter Verwendung dieser Bilder ist es egal welche Hautfarbe der Benutzer hat, was dazu führt, dass das Trainingsmaterial so auch neutraler gestaltet ist. Ziel dieser neutralität ist es die Handerkennung so zugänglicher für jeden zu machen.
Außerdem wird es notwendig werden für eine Gestenerkennung eine KI zu verwenden, um den Arbeitsaufwand vergleichsweise gering zu halten.





## 
*02.12.2020*

## Experimente mit Multithreading und weitere Verbesserung der Erkennung
*09.12.2020*

Als nun die GUI implementiert war und auch das Monitor und Kamerabild in einem Fenster und weiteren Debug-Fenstern angezeigt wurde, mussten wir feststellen, dass darunter die Performance litt.
Wir stellen fest, das die Bildwiederholrate durch die ganzen berechnungen und aktualisierungen unter 5 Bilder pro Sekunde fiel. Dies sorgte nicht nur für ein extrem ruckeliges Bild, sondern auch für eine langsamere Verarbeitung der nächsten Bilder, da das auslesen des Kamerabildes und des Monitors viel Zeit beanspruchte. Wir haben uns gedanken dazu gemacht, wie wir dieses Problem beheben können. Einfach einen immer leistungsstärkeren Computer zu verwenden kommt natürlich nicht in Frage. Also mussten wir uns was überlegen, was die benötigte Performance senkt, m die Sotware auch auf nicht so performanten Computern lauffähig zu machen, gleichzeitig aber auch sicherstellen, das alle Funktionen wie gewohnt funktionieren. Nach einer kurzen Internetrecherche kamen wir auf die Idee, einige der Aufgaben in eigene Threads auszulagern, um den Mainthread nicht unnötig zu belasten und somit ein paar Bilder fro Sekunde gutmachen können. Die einfachste Möglichkeit, die uns eingefallen ist war, das Auslesen des Kamerabildes und des Monitors in einen eigenen Thread auszulagern und das Bild in Variablen innerhalb des Objektes verfügbar zu machen um es kurz vor der Bearbeitung einfach abgreifen zu können. Dies beschwerte uns bereits einen Performancezuwachs von knapp 10-20 Bildern pro Sekunde. Dieser Unterscheid war sehr deutlich im angezeigten Videofeed dadurch zu beobachten, als dass das Video viel flüssiger war. Weitere Versuche, auch die Verarbeitung des Bildes in einen eigenen Thread auszulagern und nurnoch dann das Fenster zu aktualisieren, wenn das Bild erfolgreich verarbeitet wurde zeigten leider keine große Verbesserung.

Im Folgenden Beispiel sieht man, wie das Kamerabild automatisch immer wieder eingelesen wird wenn der Thread mittels start() gestartet wurde und innerhalb des Objektes gespeichert wird. Dies lässt sich bei bedarf einfach abrufen.


In [ ]:
class CameraGrabber(object):
    """
    Reads the Current Camera-feed in another Thread and Stores it for easy Access
    """

    def __init__(self, src, width=1280, height=720):
        """
        Initialize a new CameraGrabber
        :param src: CameraIndex from mss
        :param width: Scaled Output Image width
        :param height: Scaled Output Image hight
        """
        self.width = width
        self.height = height

        # Grab Camera Image, Resize, Convert and Store it
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, img) = self.stream.read()
        self.picture = cv2.resize(np.array(img), (self.width, self.height), interpolation=cv2.INTER_AREA)
        self.stopped = False

    def start(self):
        """
        Starts another Thread for its own get-Method, to grab the Image out of Mainloop
        :return:  Optional: The Own Object to create, start the Thread and save the Object at the same Time
        """
        Thread(target=self.get, args=()).start()
        return self

    def setSrc(self, src):
        """
        Re-Sets the Camera Input Source Index of mss
        :param src: The new Camera Index
        """
        self.stream = cv2.VideoCapture(src)

    def get(self):
        """
        Grabs the current Camera Image, Resize and stores it
        """
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, img) = self.stream.read()
                self.picture = cv2.resize(np.array(img), (self.width, self.height), interpolation=cv2.INTER_AREA)

    def stop(self):
        """
        Stops the CameraGrabber-Get-Thread started by the start-Method
        """
        self.stopped = True

Gleichzeitig wurde daran gararbeitet, die Fingererkennung noch mehr zu verfeinern. Hierzu wurden vorhandene Algorithmen weiter optimiert und Fehler wurden behoben, die das Program einfrieren ließen, wenn bestimmte Aktionen getätigt wurden. Auch wurde ein Fehler behoben, der dafür gesorgt hat, das Bilder nicht richtig übergeben wurden und somit falsch im Fenster angezeigt wurden.